# Predicting Products' Reviews

### Paine(HE Peiwei, 54471952)

### 1. Data Cleaning and Organizing

First, load the raw pickle data and calculate the mean rating of each products, then create a new dataframe with needed columns.

In [1]:
import pandas as pd
import numpy as np

In [2]:
import sklearn as sk
from sklearn import tree

In [3]:
reviews = pd.read_pickle("reviews.pickle")
counts = reviews.asin.value_counts()
reviews.overall[reviews.asin == counts.index[0]].mean()
reviews.overall[reviews.asin == counts.index[0]].value_counts()
stats = reviews.groupby(['asin'])['overall'].agg(['count', 'mean'])

In [4]:
products=pd.read_pickle("products.pickle")
products2 = products.join(stats)
products2.head()
del products2['related']
del products2['count']
del products2['brand']
del products2['imUrl']
del products2['description']
products2.head(5)

,categories,price,salesRank,title,mean
asin,,,,,
5555991584,"[[CDs & Vinyl, New Age, Celtic New Age], [CDs ...",9.49,{'Music': 939190},Memory of Trees,4.739130
B0000000ZW,"[[CDs & Vinyl, Dance & Electronic], [CDs & Vin...",23.64,{'Music': 68784},Changing Faces,4.133333
B00000016W,"[[CDs & Vinyl, Classic Rock], [CDs & Vinyl, Po...",9.49,{'Music': 77205},Pet Sounds,4.629630
B00000016T,"[[CDs & Vinyl, Alternative Rock, New Wave & Po...",14.63,{'Music': 458151},Cars,4.851064
B00000017R,"[[CDs & Vinyl, Jazz, Bebop], [CDs & Vinyl, Jaz...",8.99,{'Music': 561174},Jazz Samba,4.650000


### 2. Compute features from categories

Seperate 5 different music categories by using lambda. And create a new column of good or bad, based on the mean rating of each product, when the rating is above 4.0, would be assigned to good.

In [11]:
categories = ["Rock", "Jazz", "Dance & Electronic", "Country" ,"Classical"]

for category in categories:
    products2[category] = products2.categories.apply(lambda cats: category in cats[0])

products2= products2.rename(columns={'mean': 'mean_rating'})
products2['good'] = np.where(products2.mean_rating>4, "Good", "Bad")
    
products2.head()

,categories,price,salesRank,title,mean_rating,Rock,Jazz,Dance & Electronic,Country,Classical,good
asin,,,,,,,,,,,
5555991584,"[[CDs & Vinyl, New Age, Celtic New Age], [CDs ...",9.49,{'Music': 939190},Memory of Trees,4.739130,False,False,False,False,False,Good
B0000000ZW,"[[CDs & Vinyl, Dance & Electronic], [CDs & Vin...",23.64,{'Music': 68784},Changing Faces,4.133333,False,False,True,False,False,Good
B00000016W,"[[CDs & Vinyl, Classic Rock], [CDs & Vinyl, Po...",9.49,{'Music': 77205},Pet Sounds,4.629630,False,False,False,False,False,Good
B00000016T,"[[CDs & Vinyl, Alternative Rock, New Wave & Po...",14.63,{'Music': 458151},Cars,4.851064,False,False,False,False,False,Good
B00000017R,"[[CDs & Vinyl, Jazz, Bebop], [CDs & Vinyl, Jaz...",8.99,{'Music': 561174},Jazz Samba,4.650000,False,True,False,False,False,Good


### 3. Build a decision tree model

Split the data into test, train, and validation.

In [20]:
import sklearn
from sklearn import tree
from sklearn import cross_validation

train,test = sklearn.cross_validation.train_test_split(products2, train_size = 0.8)

clf = tree.DecisionTreeClassifier(max_depth=2)
m = clf.fit(train[categories], train.good)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [22]:
predictions = m.predict(products2[categories])
pd.crosstab(products2.good, predictions)

col_0,Good
good,
Bad,1047
Good,2521


In [8]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(products2.good, predictions))

[[   0 1047]
 [   0 2521]]


In [23]:
correct = products2.good == predictions
correct.mean()

0.70655829596412556

In [10]:
import pydotplus
from IPython.display import Image
dot_data = tree.export_graphviz(m,
  feature_names=categories, class_names=['bad', 'good'], 
  filled=True, rounded=True)  
graph = pydotplus.graph_from_dot_data(dot_data)  

SyntaxError: unexpected EOF while parsing (<ipython-input-10-d195ab179a70>, line 6)

### 4. Test the decision tree model

In [24]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=2)
m = clf.fit(test[categories], test.good)

In [25]:
predictions = m.predict(products2[categories])
pd.crosstab(products2.good, predictions)

col_0,Good
good,
Bad,1047
Good,2521


In [26]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(products2.good, predictions))

[[   0 1047]
 [   0 2521]]
